In [336]:
import os 
import numpy as np 

In [ ]:
# os.system(scenedetect -i "input.avi" -o "...\Computer Vision\project\Dataset\keyframe" detect-threshold -t 50 split-video list-scenes)

ConvFeats Feature Extraction 

In [337]:
from tensorflow import keras
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.layers import MaxPool2D, Flatten, Dense
from tensorflow.keras import Model

In [333]:

def convfeats(image):
    image = img_to_array(load_img(image, target_size=(224, 224)))

    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

    # prepare the image for the VGG model
    image = preprocess_input(image)

    # 1st Conv Block

    x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(image)
    x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(x)
    x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
    # 2nd Conv Block

    x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
    x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
    x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
    # 3rd Conv block

    x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
    x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
    x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
    x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
    # 4th Conv block

    x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
    x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
    x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
    x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)

    # 5th Conv block

    x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
    x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
    x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
    x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
    # Fully connected layers

    x = Flatten()(x)

    return Dense(units = 4096, activation ='relu')(x)

In [335]:
image = os.getcwd() + '/Dataset/keyframes/big_buck_bunny_480p_stereo-Scene-004-02.jpg'

In [334]:
convFeats = keras.layers.Concatenate(axis=0)([convFeats,k_new])

NameError: name 'convFeats' is not defined

CSIFT Feature Extraction

In [215]:
import cv2
import os 
import skimage.io as io 
from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids
image = (os.getcwd() + "/Dataset/keyframes/big_buck_bunny_480p_stereo-Scene-007-02.jpg").replace("\\","/")
# image = io.imread(image)
image = cv2.imread(image,1)
sift = cv2.xfeatures2d.SIFT_create()
(key_points,descriptors) = sift.detectAndCompute(image, None)


In [216]:
points = [key_points[idx].pt for idx in range(0, len(key_points))]


In [217]:
kmean = KMeans(100)

In [218]:
kmean.fit(points)

KMeans(n_clusters=100)

In [219]:
clustered_points = kmean.cluster_centers_

In [220]:
backtorgb = cv2.cvtColor(descriptors,cv2.COLOR_GRAY2RGB)
descriptor_in_rgb = backtorgb.reshape(backtorgb.shape[0],backtorgb.shape[1]*backtorgb.shape[2])
kmedoids = KMedoids(n_clusters=100, random_state=0,metric='euclidean').fit(descriptor_in_rgb)

In [221]:
kmedoids_selection = kmedoids.cluster_centers_

In [222]:
CSIFTFeatures = clustered_points,kmedoids_selection

MFCC Features

In [224]:
from scipy.io import wavfile
from python_speech_features import mfcc, logfbank

# ffmpeg -i input.mp4 -vn -acodec pcm_s16le -ar 16000 -ac 2 output.wav

frequency_sampling, audio_signal = wavfile.read("Dataset/keyframe_audio/output.wav")

features_mfcc = mfcc(audio_signal, frequency_sampling)




(22949, 13)

In [225]:
kmean_for_audio = KMeans(500)

In [226]:
kmean_for_audio.fit(features_mfcc)

KMeans(n_clusters=500)

In [315]:
clustered_audio_samples = kmean_for_audio.cluster_centers_
clustered_audio_samples.shape

(500, 13)

MFCCfeature ---> CNN

In [313]:
import tensorflow as tf
from tensorflow.keras import layers, models

def cnn_for_mfcc(input):
    model = models.Sequential()
    model.add(layers.Conv1D(13, 32, activation = 'relu', input_shape=input.shape))
    model.add(layers.MaxPooling1D(6))

    model.add(layers.Conv1D(32, 64,activation = 'relu'))
    model.add(layers.MaxPooling1D(7))

    model.add(layers.Conv1D(64, 128 , activation = 'relu'))
    model.add(layers.MaxPooling1D(8))
    model.add(Flatten())
    model.summary()
    return model(input)

Model: "sequential_72"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_130 (Conv1D)          (None, 469, 13)           5421      
_________________________________________________________________
max_pooling1d_99 (MaxPooling (None, 78, 13)            0         
_________________________________________________________________
conv1d_131 (Conv1D)          (None, 15, 32)            26656     
_________________________________________________________________
max_pooling1d_100 (MaxPoolin (None, 2, 32)             0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 64)                0         
Total params: 32,077
Trainable params: 32,077
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def cnn_for_convfeats(input):
    model = models.Sequential()
    model.add(layers.Conv1D(4096, 1024, activation = 'relu', input_shape=input.shape))
    model.add(layers.MaxPooling1D(2))

    model.add(layers.Conv1D(1024, 256,activation = 'relu'))
    model.add(layers.MaxPooling1D(2))

    model.add(Flatten())
    return model(input)

In [316]:
def cnn_for_csift(input):
    model = models.Sequential()
    model.add(layers.Conv1D(384, 300, activation = 'relu', input_shape=input.shape))
    model.add(layers.MaxPooling1D(4))

    model.add(layers.Conv1D(300, 256,activation = 'relu'))
    model.add(layers.MaxPooling1D(4))

    model.add(layers.Conv1D(256, 128 , activation = 'relu'))
    model.add(layers.MaxPooling1D(5))
    model.add(Flatten())
    return model(input)

LSTM Architecture 

In [327]:
from keras.models import Sequential
from keras.layers import CuDNNLSTM, Dense, Dropout, LSTM
from keras.activations import sigmoid 

def lstm_model(concatenated_input):
    classifier = Sequential()
    classifier.add(CuDNNLSTM(1, input_shape=(concatenated_input.shape), return_sequences=True))
    classifier.add(Dropout(0.2))
    return sigmoid(classifier(concatenated_input))